In [1]:
import spacy
import string
import numpy as np
import itertools
from tqdm import tqdm

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from alibi.explainers import AnchorText
from alibi.datasets import fetch_movie_sentiment
from alibi.utils.download import spacy_model
from alibi.utils.lang_model import DistilbertBaseUncased, BertBaseUncased, RobertaBase

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"


%load_ext autoreload
%autoreload 2

/home/robert/anaconda3/envs/alibi/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


### Load movie review dataset

The `fetch_movie_sentiment` function returns a `Bunch` object containing the features, the targets and the target names for the dataset.

In [2]:
movies = fetch_movie_sentiment()
movies.keys()

dict_keys(['data', 'target', 'target_names'])

In [3]:
data = movies.data
labels = movies.target
target_names = movies.target_names

In [4]:
train, test, train_labels, test_labels = train_test_split(data, labels, test_size=.2, random_state=42)
train, val, train_labels, val_labels = train_test_split(train, train_labels, test_size=.1, random_state=42)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
val_labels = np.array(val_labels)

### Apply CountVectorizer to training set

In [5]:
vectorizer = CountVectorizer(min_df=1)
vectorizer.fit(train)

CountVectorizer()

### Fit model

In [6]:
np.random.seed(0)
clf = LogisticRegression(solver='liblinear')
clf.fit(vectorizer.transform(train), train_labels)

LogisticRegression(solver='liblinear')

### Define prediction function

In [7]:
predict_fn = lambda x: clf.predict(vectorizer.transform(x))

### Make predictions on train and test sets

In [8]:
preds_train = predict_fn(train)
preds_val = predict_fn(val)
preds_test = predict_fn(test)
print('Train accuracy', accuracy_score(train_labels, preds_train))
print('Validation accuracy', accuracy_score(val_labels, preds_val))
print('Test accuracy', accuracy_score(test_labels, preds_test))

Train accuracy 0.9801624284382905
Validation accuracy 0.7544910179640718
Test accuracy 0.7589841878294202


### Load spaCy model

English multi-task CNN trained on OntoNotes, with GloVe vectors trained on Common Crawl. Assigns word vectors, context-specific token vectors, POS tags, dependency parse and named entities.

In [9]:
model = 'en_core_web_md'
spacy_model(model=model)
nlp = spacy.load(model)

### Load transformers

In [10]:
lang_model = DistilbertBaseUncased()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


### Random indices

In [11]:
indices = np.random.choice(len(test), size=10, replace=False)
print(indices)

[ 299  379  997 1601  264  229 1774 1968  458  252]


### Define Anchor

In [14]:
config = {
    "top_n": 50,
    "sample_proba": 0.5,
    "filling_method": 'parallel',
    "prec_mask_templates": 0.1,
    "punctuation": string.punctuation
}

explainer_sim = AnchorText(nlp=nlp, sampling_method="similarity", predictor=predict_fn, **config)
explainer_lm = AnchorText(language_model=lang_model, sampling_method="language_model", predictor=predict_fn, **config)

The following keys are incorrect: punctuation, prec_mask_templates, filling_method
The following keys are incorrect: prec_mask_templates


In [15]:
def build_explanation(explanation) -> str:
    s = ''
    s += 'Anchor: %s\n' % (' AND '.join(explanation.anchor))
    s += 'Precision: %.2f\n' % explanation.precision
    
    # print examples covered as True
    s += '\n\nExamples where anchor applies and model predicts %s:\n' % pred
    if len(explanation.raw['examples']):
        s += '\n'.join([x for x in explanation.raw['examples'][-1]['covered_true']])
    
    # print examples covered as False
    s += '\n\nExamples where anchor applies and model predicts %s:\n' % alternative
    if len(explanation.raw['examples']):
        s += '\n'.join([x for x in explanation.raw['examples'][-1]['covered_false']])
    
    s += '\n\n\n'
    return s

In [16]:
class_names = movies.target_names

### Example 0

In [18]:
text = test[indices[0]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer_sim.explain(text, threshold=0.95)

# language model explanation
lm_explanation = explainer_lm.explain(text, threshold=0.95)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [19]:
print("Original text:", text)

Original text: a fantastically vital movie that manages to invest real humor , sensuality , and sympathy into a story about two adolescent boys .


In [20]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: manages AND about
Precision: 0.95


Examples where anchor applies and model predicts positive:
no fantastically vital episode whatever manages to aquire financial humor , sensuality , and animosity into another story about two adolescent boys .
a fantastically considerable porno that manages to grow real humor , depravity , and fondness into any narrative about two adolescent boys .
an fantastically vital scene that manages to invest real humor , sensuality , and sympathy across some story about two adolescent boys .
the depressingly vital movie that manages to invest real humor , fleshy , and sympathy within any story about two asian youths .
any fantastically vital movie that manages to invest real absurdity , altruism , and ire into the writer about two adolescent ladies .
this enormously responsible cinema that manages to continue real humour , cowardice , and fondness into each fable about two adolescent chaps .
a fantastically vital movie which manages to lend ultimate co

In [21]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: manages AND two
Precision: 0.97


Examples where anchor applies and model predicts positive:
a fantastically vital movie serial manages to invest real humor, morality, imp comical adventure of fantasies involving two adolescent boys.
a fantastically vital movie show manages to invest within humor, heroism, im humor sentimental hero scenes towards two adolescent boys.
a fantastically vital movie character manages to invest amidst humor, imagination, im qui and con stories portraying two adolescent boys.
a fantastically vital movie character manages to invest her humor, fantasy, and nu qui emotional while and two adolescent boys.
a fantastically vital movie poster manages to invest its humor, ambition, und em of romance stunts entertaining two adolescent boys.
a fantastically vital movie maker manages to invest out humor, funny, daring funny the con jokes featuring two adolescent boys.
a fantastically vital movie company manages to invest beyond humor, magic, drama moral et or wh

### Example 1

In [22]:
text = test[indices[1]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer_sim.explain(text)

# language model explanation
lm_explanation = explainer_lm.explain(text)

/home/robert/workspace/alibi/alibi/explainers/anchor_text.py:116: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  by_similarity = sorted(queries, key=lambda w: word_vocab.similarity(w), reverse=True)[:self.n_similar]
/home/robert/workspace/alibi/alibi/explainers/anchor_text.py:127: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  similarities.append(word_vocab.similarity(lexeme))


In [23]:
print("Original text:", text)

Original text: there is a refreshing absence of cynicism in stuart little 2--quite a rarity , even in the family film market . eventually , it wins you over .


In [24]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: refreshing AND family AND even AND film AND market AND absence AND in
Precision: 0.95


Examples where anchor applies and model predicts positive:
there is an refreshing absence of cynicism on stuart shy 2 - level a occurance , even in each family film market . eventually , it sees you out .
there is a refreshing absence because cynicism in australian own 2 - -quite each greatness , even in these family film market . eventually , it selects you out .
there is a refreshing absence with hostility in stuart good 2 - -quite any craftsmanship , even in the family film market . even , it wins you over .
there is each refreshing absence throughout cynicism although irish little 2 - anything both jewel , even in the family film market . eventually , it wins you out .
there is every refreshing absence during cynicism midst irish tiny 2 - side the popularity , even in the family film market . mysteriously , it bids you over .
there is any refreshing absence whether helplessness over scot

In [25]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: refreshing AND film AND family AND market AND you AND rarity AND absence AND it
Precision: 0.95


Examples where anchor applies and model predicts positive:
there is his refreshing absence of cynicism aboard stuart little appears - - quite into rarity, even from the family film market. yes, it wants you funny.
there is always refreshing absence of cynicism or stuart little faces - - quite below rarity, even entering the family film market. yet, it pulls you forget.
there is fairly refreshing absence of cynicism behind stuart little works - - quite of rarity, even through the family film market. so, it reminds you understand.
there is the refreshing absence of cynicism under stuart little guy - - quite their rarity, even concerning the family film market. moreover, it seems you both.
there is the refreshing absence of cynicism within stuart little eyes - - quite much rarity, even upon the family film market. oh, it draws you laugh.
there is such refreshing absence of cynicism wi

### Example 2

In [26]:
text = test[indices[2]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer_sim.explain(text)

# language model explanation
lm_explanation = explainer_lm.explain(text)

Could not find an result satisfying the 0.95 precision constraint. Now returning the best non-eligible result.


In [27]:
print("Original text:", text)

Original text: it doesn't offer audiences any way of gripping what its point is , or even its attitude toward its subject .


In [28]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: offer AND point AND audiences AND toward
Precision: 0.63


Examples where anchor applies and model predicts negative:
it does n't offer audiences some trouble with gripping what its point is , or even its ethic toward its individual .
it does n't offer audiences any way after touching what its point is , or however its politeness toward its anyone .
it does n't offer audiences any way of motivating what its point is , or even its stance toward its subject .
it does n't offer audiences those thing of gripping what its point is , or possibly its attitude toward its interpretation .
it does n't offer audiences both situation into gripping what its point is , or then its attitude toward its explaination .
it does n't offer audiences any guy within motivating what its point is , or unfortunately its attitude toward its subject .
it does n't offer audiences any whole into empowering what its point is , or even its attitude toward its liability .
it does n't offer audiences every trut

In [29]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: doesn AND point AND offer AND its AND subject
Precision: 0.96


Examples where anchor applies and model predicts negative:
god doesn't offer it any ease of gripping what its point does, without even it to examine its subject.
reason doesn't offer scholars any intentions of gripping what its point was, preferring even merely it debating its subject.
literature doesn't offer customers any amount of gripping what its point causes, preferring even as arguments by its subject.
socrates doesn't offer about any notion of gripping what its point intends, preferring even further opinion describes its subject.
evidence doesn't offer users any type of gripping what its point has, nobody even considers it raises its subject.
pbs doesn't offer himself any intentions of gripping what its point makes, rarely even considering questions around its subject.
rhetoric doesn't offer students any satisfaction of gripping what its point was, rather even openly directly examines its subject.
documenta

### Example 3

In [43]:
text = test[indices[3]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer_sim.explain(text)

# language model explanation
lm_explanation = explainer_lm.explain(text)

In [44]:
print("Original text:", text)

Original text: rodriguez . . . was unable to reproduce the special spark between the characters that made the first film such a delight .


In [45]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: between AND rodriguez AND unable
Precision: 0.95


Examples where anchor applies and model predicts negative:
rodriguez . . . was unable to reproduce the special spark between this voices that made any forthcoming footage such a delight .
rodriguez . . . was unable to reproduce the possible curiosity between the figures which made the first movie such this delight .
rodriguez . . . was unable to reproduce the special spark between those personalities whatever had the previous trilogy such the amusement .
rodriguez . . . was unable to reproduce both special spark between a characters whatever made the first film such a beauty .
rodriguez . . . was unable to interpret both extraordinary spark between those characters which supposed any same film such this delight .
rodriguez . . . was unable to reproduce the fantastic spark between each characters that made the little film such the enthusiasm .
rodriguez . . . was unable to emulate the festive spark between the characters that ma

In [46]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: to AND was AND spark AND unable AND characters AND the
Precision: 0.98


Examples where anchor applies and model predicts negative:
rodriguez... was unable to produce his special spark between the characters having made no first script show between him.
rodriguez... was unable to discover enough special spark between the characters ever made another first series against radio returns.
rodriguez... was unable to detect any special spark between the characters when made this first character cast about movies.
rodriguez... was unable to share clear special spark between the characters because made to first episode on clint vargas.
rodriguez... was unable to extract without special spark between the characters had made him first feature through her again.
rodriguez... was unable to convey strong special spark between the characters unless made your first movie of comic story.
rodriguez... was unable to see their special spark between the characters than made this first person fan r

### Example 4

In [47]:
text = test[indices[4]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer_sim.explain(text)

# language model explanation
lm_explanation = explainer_lm.explain(text)

In [48]:
print("Original text:", text)

Original text: while the story's undeniably hard to follow , iwai's gorgeous visuals seduce .


In [49]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: follow AND undeniably AND gorgeous
Precision: 0.97


Examples where anchor applies and model predicts positive:
while the epilogue becomes undeniably tough to follow , iwai 's gorgeous visuals hug .
while a story takes undeniably big to follow , iwai 's gorgeous visuals hug .
while the story 's undeniably hard to follow , iwai 's gorgeous visuals hug .
while the excerpt is undeniably hard to follow , iwai 's gorgeous visuals seduce .
while these story 's undeniably hard to follow , iwai 's gorgeous visuals seduce .
while every story holds undeniably stupid to follow , iwai 's gorgeous visuals seduce .
while the story 's undeniably fucking to follow , iwai 's gorgeous visuals saucy .
while any story has undeniably dirty to follow , iwai 's gorgeous visuals hug .
while the story asks undeniably tightest to follow , iwai 's gorgeous moments hug .
while each character trys undeniably soft to follow , iwai 's gorgeous imaginations seduce .

Examples where anchor applies and model pr

In [50]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: follow AND seduce
Precision: 0.96


Examples where anchor applies and model predicts positive:
while the film'music undeniably gorgeous messages follow, clark's gorgeous visuals seduce.
while the others'backgrounds undeniably bizarre narratives follow, annie's gorgeous visuals seduce.
while the shows'backgrounds undeniably passionate numbers follow, jackson's gorgeous visuals seduce.
while the music'stunning undeniably pleasing photos follow, naomi's gorgeous visuals seduce.
while the videos'seemingly undeniably provocative pictures follow, leo's gorgeous visuals seduce.
while the film'beautiful undeniably compelling effects follow, nina's gorgeous visuals seduce.
while the episode'seem undeniably vivid pictures follow, gaga's gorgeous visuals seduce.
while the tracks'capt undeniably intricate motifs follow, dante's gorgeous visuals seduce.
while the tracks'trademark undeniably energetic interiors follow, rihanna's gorgeous visuals seduce.
while the sequels'increasingly undenia

### Example 5

In [51]:
text = test[indices[5]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer_sim.explain(text)

# language model explanation
lm_explanation = explainer_lm.explain(text)

In [52]:
print("Original text:", text)

Original text: i have two words to say about reign of fire . great dragons !


In [53]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: dragons AND have AND fire
Precision: 0.95


Examples where anchor applies and model predicts negative:
i have two phrases to say about reign of fire . great dragons !
i have two words to say far reign of fire . cool dragons !
i have two words to let now mediocrity into fire . impressive dragons !
i have two words to guess about reign of fire . perfect dragons !
i have two words to bother about downfall of fire . easy dragons !
i have two words to guess somewhere dictatorship that fire . important dragons !
i have two sentences to do sometimes predecessor under fire . great dragons !
i have two voices to worry about reign among fire . brilliant dragons !
i have two scriptures to admit rather vengeance of fire . little dragons !
i have two words to say just millenia of fire . great dragons !

Examples where anchor applies and model predicts positive:
i have two gospels to know though reign of fire . remarkable dragons !
i have two words to feel about harem of fire . happy dragons

In [54]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: dragons AND fire AND say
Precision: 0.95


Examples where anchor applies and model predicts negative:
i said two words to say the reign of fire. great dragons!
i waited two words to say i reign of fire. great dragons!
i needed two words to say concerning reign of fire. great dragons!
i find two words to say — reign of fire. great dragons!
i took two words to say hail reign of fire. great dragons!
i expected two words to say bloody reign of fire. great dragons!
i have two words to say o reign of fire. great dragons!
fire dared enough started to say that go full fire. great dragons!
th no went have to say great fire is fire. great dragons!
brave said are heard to say go a holy fire. great dragons!

Examples where anchor applies and model predicts positive:
i exchanged two words to say dear reign of fire. great dragons!
i chose two words to say against reign of fire. great dragons!
i want two words to say i reign of fire. great dragons!
you we it how to say ae sword and fire. grea

### Example 6

In [55]:
text = test[indices[6]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer_sim.explain(text)

# language model explanation
lm_explanation = explainer_lm.explain(text)

In [56]:
print("Original text:", text)

Original text: what's next : " my mother the car ? "


In [57]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: car AND next
Precision: 0.99


Examples where anchor applies and model predicts negative:
what 's next : " my friend a car ? "
what 's next : " my child the car ? "
what 's next : " my year the car ? "
what 's next : " my grandchild an car ? "
what 's next : " my grandparent every car ? "
what 's next : " my birth any car ? "
what 's next : " my mother the car ? "
what 's next : " my cousin the car ? "
what 's next : " my mama an car ? "
what 's next : " my babysit the car ? "

Examples where anchor applies and model predicts positive:
what 's next : " my family both car ? "
what 's next : " my baby an car ? "
what 's next : " my child these car ? "





In [58]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: next
Precision: 0.96


Examples where anchor applies and model predicts negative:
mike'the next : " my or the car? "
they'song next : " my ride the car? "
evans'll next : " my gran the car? "
mike'07 next : " my grandmother the car? "
he'say next : " my girlfriend the car? "
williams'the next : " my problem the car? "
i're next : " my boy the car? "
evans'next next : " my name the car? "
goin'cause next : " my aunt the car? "
peter's next : " my little no you? "

Examples where anchor applies and model predicts positive:
dave'round next : " my lady the car? "
drake's next : " my first wedding here? "
questions'right next : " my mother told now? "
what'about next : " my mother beat mommy? "
what'rec next : " my mother killed divorced? "
cbs'ring next : " you mother beat not? "





### Example 7

In [59]:
text = test[indices[7]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer_sim.explain(text)

# language model explanation
lm_explanation = explainer_lm.explain(text)

In [60]:
print("Original text:", text)

Original text: supposedly based upon real , or at least soberly reported incidents , the film ends with a large human tragedy . alas , getting there is not even half the interest .


In [33]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: getting
Precision: 0.98


Examples where anchor applies and model predicts negative:
technically oriented upon huge , or than least soberly taken allegations , all installment ends along every entire latent tragedy . alas , getting there is not only half those borrower .
even written upon incredible , or around least soberly published incidents , the film means with another large human disaster . alas , getting there is not then half another extent .
supposedly compared upon strong , or into worst soberly ascribed consequences , the movie follows around every large humankind tragedy . alas , getting there is not often half the interest .
practically focussed upon good , or at worst happily collected accidents , those director ends like a ample knowledge outrage . alas , getting there is not too half another interest .
ultimately based upon perfect , or to worst thoughtfully conducted incidents , this slapstick ends despite a large latent hopelessness . alas , getting there is n

In [34]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: getting AND supposedly AND there AND incidents AND alas
Precision: 0.97


Examples where anchor applies and model predicts negative:
supposedly following for observations, or at other soberly reported incidents, the incident invariably to a tragic human tragedy. alas, getting there usually not quite further the interest.
supposedly concerning involving individuals, or at all soberly reported incidents, the tragic generally as a genuinely human tragedy. alas, getting there ought not further provoke the interest.
supposedly regarding those crimes, or at rarely soberly reported incidents, the police actually be a natural human tragedy. alas, getting there must not precisely spread the interest.
supposedly without or emergencies, or at present soberly reported incidents, the police have creates a huge human tragedy. alas, getting there probably not totally boost the interest.
supposedly about with details, or at others soberly reported incidents, the weather always witnessing a hor

### Example 8

In [35]:
text = test[indices[8]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer_sim.explain(text)

# language model explanation
lm_explanation = explainer_lm.explain(text)

In [36]:
print("Original text:", text)

Original text: a well-crafted letdown .


In [37]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: crafted AND well AND a
Precision: 0.97


Examples where anchor applies and model predicts positive:
a well - crafted letdown .
a well - crafted letdown .
a well - crafted letdown .
a well - crafted unhappiness .
a well - crafted letdown .
a well - crafted letdown .
a well - crafted letdown .
a well - crafted letdown .
a well - crafted letdown .
a well - crafted desperation .

Examples where anchor applies and model predicts negative:
a well - crafted embarrassment .
a well - crafted disappointment .
a well - crafted embarrassment .





In [38]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: crafted AND well AND a
Precision: 0.98


Examples where anchor applies and model predicts positive:
a well - crafted tale.
a well - crafted bow.
a well - crafted crossbow.
a well - crafted doll.
a well - crafted necklace.
a well - crafted recipe.
a well - crafted toy.
a well - crafted design.
a well - crafted tapestry.
a well - crafted toy.

Examples where anchor applies and model predicts negative:
a well - crafted script.
a well - crafted script.





### Example 9

In [39]:
text = test[indices[9]]

# compute text prediction
pred = class_names[predict_fn([text])[0]]
alternative = class_names[1 - predict_fn([text])[0]]

# similarity explanation
sim_explanation = explainer_sim.explain(text)

# language model explanation
lm_explanation = explainer_lm.explain(text)

In [40]:
print("Original text:", text)

Original text: it's just hard to believe that a life like this can sound so dull .


In [41]:
# display similarity explanation
print(build_explanation(sim_explanation))

Anchor: dull
Precision: 0.99


Examples where anchor applies and model predicts negative:
it 's just hard to believe that no joy like this can sound so dull .
it 's so stiff to argue that a life like a can drum so dull .
it 's not hard to believe that each moment if an can sound so dull .
it 's down hard to agree that a embark like this can flamenco actually dull .
it 's scarcely hard to guess that a life like this can sound so dull .
it 's not nasty to believe that every life along each can sound so dull .
it 's just hard to beleive that an life past this can feel rather dull .
it 's obviously hard to believe that a life like this can sound probably dull .
it 's just massive to remeber that every life without the can drum so dull .
it 's down soft to disagree that a soul as any can sound so dull .

Examples where anchor applies and model predicts positive:
it 's kind solid to believe that any voyage inside this can horn anyway dull .





In [42]:
# display language model explanation
print(build_explanation(lm_explanation))

Anchor: like
Precision: 0.98


Examples where anchor applies and model predicts negative:
it'r more hard everyone believe that a life like grandma boring seems so dull.
it'z s hard on believe into a life like this you nothing so dull.
it'an more hard ya believe living a life like jack rarely times so dull.
it'g sometimes hard couples believe from a life like la and was so dull.
it'b got hard he believe at a life like ours is or so dull.
it'e more hard nobody believe experiencing a life like an this seem so dull.
it'ts painfully hard friends believe was a life like you seems looks so dull.
it'wasn probably hard readers believe inside a life like la that feel so dull.
it'sis pretty hard and believe just a life like this day it so dull.
it'ds incredibly hard in believe finding a life like another are being so dull.

Examples where anchor applies and model predicts positive:
dave's truly hard chicks believe taking a stuff like this can sound so crazy.
today's eyes ability to keep different